In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords # the for of in with
from nltk.stem.porter import PorterStemmer # loved loving == love
from sklearn.feature_extraction.text import TfidfVectorizer # loved = [0.0]
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
news_df = pd.read_csv('train.csv', error_bad_lines=False, engine="python")

<ipython-input-37-3ab951bfa306>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  news_df = pd.read_csv('train.csv', error_bad_lines=False, engine="python")
Skipping line 13390: unexpected end of data


In [ ]:
news_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
news_df.shape


(13388, 5)

In [ ]:
news_df.isna().sum()


id           0
title      355
author    1253
text        30
label        0
dtype: int64

In [ ]:
news_df = news_df.fillna(' ')

In [ ]:
news_df.isna().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [ ]:
news_df['content'] = news_df['author']+" "+news_df['title']

In [ ]:
news_df

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...,...
13383,13383,Are You Ready for Aftermath of the Election? W...,Dave Hodges,Previous Are You Ready for Aftermath of the El...,1,Dave Hodges Are You Ready for Aftermath of the...
13384,13384,Mysterious 'pinging' at bottom of Arctic sea.....,,\nUK TELEGRAPH \nCanada's military is investig...,1,Mysterious 'pinging' at bottom of Arctic sea...
13385,13385,"Edgar Feuchtwanger Bore Witness, Horribly Clos...",William Grimes,To be a Jew in Germany in the 1930s was not co...,0,William Grimes Edgar Feuchtwanger Bore Witness...
13386,13386,Hillary Reveals Her Pick for Secretary of Stat...,The Conservative Millennial,0 comments \nHillary Clinton has reportedly be...,1,The Conservative Millennial Hillary Reveals He...


In [ ]:
news_df['content']

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
13383    Dave Hodges Are You Ready for Aftermath of the...
13384      Mysterious 'pinging' at bottom of Arctic sea...
13385    William Grimes Edgar Feuchtwanger Bore Witness...
13386    The Conservative Millennial Hillary Reveals He...
13387    Thomas Erdbrink Long Avoided by Tourists, Iran...
Name: content, Length: 13388, dtype: object

In [ ]:
# stemming
ps = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
news_df['content'] = news_df['content'].apply(stemming)

In [ ]:
news_df['content']

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
13383                dave hodg readi aftermath elect elect
13384    mysteri ping bottom arctic sea wildlif spook r...
13385    william grime edgar feuchtwang bore wit horrib...
13386    conserv millenni hillari reveal pick secretari...
13387    thoma erdbrink long avoid tourist iran suddenl...
Name: content, Length: 13388, dtype: object

In [ ]:
X = news_df['content'].values
y = news_df['label'].values

In [ ]:
print(X)
print(y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'william grime edgar feuchtwang bore wit horribl close hitler new york time'
 'conserv millenni hillari reveal pick secretari state woman cannot elect'
 'thoma erdbrink long avoid tourist iran suddenli hot destin new york time']
[1 0 1 ... 0 1 0]


In [ ]:
vector = TfidfVectorizer()
vector.fit(X)
X = vector.transform(X)

In [ ]:
print(X)

  (0, 12892)	0.28917314212218276
  (0, 11065)	0.2636271674324723
  (0, 7332)	0.3613167752423448
  (0, 7108)	0.2901825471572018
  (0, 6352)	0.24374094706910188
  (0, 5798)	0.22259560635941825
  (0, 4126)	0.2355752033270912
  (0, 3160)	0.2727890125236489
  (0, 2997)	0.3613167752423448
  (0, 2466)	0.24772587837763652
  (0, 2068)	0.35626414982781024
  (0, 237)	0.2727890125236489
  (1, 13776)	0.2993483277822986
  (1, 5646)	0.18927905094323183
  (1, 4574)	0.715157361351818
  (1, 2971)	0.26618488679597996
  (1, 2340)	0.19020205594120276
  (1, 1857)	0.3854876227809506
  (1, 1590)	0.15329813541333714
  (1, 1266)	0.28979600816938966
  (2, 12835)	0.4229544554329089
  (2, 7897)	0.4873853559765643
  (2, 4948)	0.3480112177398494
  (2, 4482)	0.38213572550143815
  (2, 2587)	0.4625151198757147
  :	:
  (13385, 3778)	0.3941960874818855
  (13385, 2350)	0.25887452403128813
  (13385, 1485)	0.3500461404517771
  (13386, 13776)	0.3142163971297095
  (13386, 11883)	0.251364842091813
  (13386, 11057)	0.3516781346

In [ ]:
print(y)

[1 0 1 ... 0 1 0]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=1)

In [ ]:
X_train.shape

(10710, 14037)

In [ ]:
X_test.shape

(2678, 14037)

In [ ]:
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

In [ ]:
train_y_pred = model.predict(X_train)
print("train accurracy :",accuracy_score(train_y_pred,y_train))

train accurracy : 0.9838468720821661


In [ ]:
test_y_pred = model.predict(X_test)
print("train accurracy :",accuracy_score(test_y_pred,y_test))

train accurracy : 0.9716206123973115


In [ ]:
# prediction system

input_data = X_test[1]
prediction = model.predict(input_data)
if prediction[0] == 1:
    print('Fake news')
else:
    print('Real news')

Real news


In [ ]:
news_df['content'][4]

'howard portnoy iranian woman jail fiction unpublish stori woman stone death adulteri'

In [ ]:
news_df['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'